In [37]:
import pandas as pd
import numpy as np
from create_sCASP_files import *



# 01

In [381]:
norm = 'L2'

In [382]:
model = 'MC3G'
#model = 'C3G'

scasp_path = './s(CASP)_'+model+'/'
#scasp_path = './s(CASP)'

In [383]:

train_path = './Data/train_fold_1.csv'
test_path = './Data/test_fold_1.csv'

# Index of the source instance in the test dataset. We find the counterfactual of this instance 
ref_index=2

In [384]:
k = 20

In [385]:
df_org = pd.read_csv(train_path)

df_org[:6]

,Buying,Maint,Doors,Persons,Lugboot,Safety,Label
0,vhigh,vhigh,morethanfive,more,big,high,negative
1,med,vhigh,3,4,small,med,negative
2,low,low,morethanfive,2,big,high,negative
3,high,med,4,2,med,med,negative
4,high,med,3,2,med,low,negative
5,high,med,4,more,small,high,positive


In [386]:
df_org.columns

Index(['Buying', 'Maint', 'Doors', 'Persons', 'Lugboot', 'Safety', 'Label'], dtype='object')

In [387]:
### Select the features that contribute to the prediction
Feature_Columns = ['Persons', 'Safety','Buying', 'Maint']
print(Feature_Columns)

['Persons', 'Safety', 'Buying', 'Maint']


In [388]:
df_ref = pd.read_csv(test_path)
df_fact = df_ref.copy()[Feature_Columns]

In [389]:
df_fact.iloc[ref_index]

Persons       2
Safety      med
Buying     high
Maint      high
Name: 2, dtype: object

In [390]:
df_org_1 = df_org[Feature_Columns].copy()
#df_org_1

In [391]:
df_fact.iloc[ref_index]

Persons       2
Safety      med
Buying     high
Maint      high
Name: 2, dtype: object

In [392]:
# Obtain a dictionary with domain information about features in 'Feature_Columns
info_dict = build_feature_info(df_org_1, Feature_Columns)
#info_dict = build_feature_info_1(df_org_1, Feature_Columns, df_ref.iloc[ref_index], 10)
print(info_dict)

{'Persons': ['more', '4', '2'], 'Safety': ['high', 'med', 'low'], 'Buying': ['vhigh', 'med', 'low', 'high'], 'Maint': ['vhigh', 'low', 'med', 'high']}


In [393]:
# Write Domain information to a s(CASP) file for the s(CASP) code
write_domains(info_dict, scasp_path+"Domain_7.pl")
# Write Property information to a s(CASP) file for the s(CASP) code
write_properties(Feature_Columns, scasp_path+"Property_7.pl")

Domain facts written to ./s(CASP)_MC3G/Domain_7.pl
Wrote all feature lines to ./s(CASP)_MC3G/Property_7.pl


In [394]:
# Obtain the information of the source sample that we need a counterfactual of
# Store this information in a s(CASP) file as we need to create the counterfactual


# Print the source to view its attributes
print(df_fact.iloc[ref_index])
# Write this sample to the s(CASP) file
write_original_sample_auto(df_fact, ref_index, scasp_path+"Original_sample.pl", Feature_Columns, 'original_sample')

Persons       2
Safety      med
Buying     high
Maint      high
Name: 2, dtype: object
Wrote predicate to ./s(CASP)_MC3G/Original_sample.pl


In [395]:
df_fact[:6]

,Persons,Safety,Buying,Maint
0,2,high,high,high
1,4,med,med,low
2,2,med,high,high
3,4,med,low,high
4,more,low,med,low
5,4,med,med,vhigh


In [396]:
# Obtain the information of the mutability of the requied features
# 'Z'- no constraint
# 0 - Stay constant i.e. immutable
# 1 - change (for categorical features)/ increase (for numeric features)
# -1 - decrease (for numeric features)

mutability_constraints = ['Z','Z','Z', 'Z']

write_mutability_predicate(
    mutability_list=mutability_constraints,
    output_file=scasp_path+"Mutability.pl",
    predicate_name="mutability")

Wrote predicate to /Users/spam/Documents/UTD/PHD/ALPS/Spring_25/NeSY_Late_Deadline/Check_Datasets_cars/s(CASP)_MC3G/Mutability.pl


In [397]:
# Obtain the information of the mutability of the requied features
# 'W': no weight - undesired
# number- given weight o the number
weight_constraints = [1,1,1,1]

write_weight_predicate(
    weight_list=weight_constraints,
    output_file=scasp_path+"Weight.pl",
    predicate_name="weight"
)


Wrote predicate to /Users/spam/Documents/UTD/PHD/ALPS/Spring_25/NeSY_Late_Deadline/Check_Datasets_cars/s(CASP)_MC3G/Weight.pl


# 02

In [398]:
from sCASP_subprocess import *



In [399]:
scasp_path

'./s(CASP)_MC3G/'

In [400]:
scasp_opt_file = "cf_7.txt"
scasp_file = "test_2.pl"


run_sCASP_script(scasp_opt_file, scasp_file,scasp_path+'/')

Output:
 % QUERY:?- original_sample(F_Persons,F_Safety,F_Buying,F_Maint),mutability(Z1,Z2,Z3,Z4),weight(W1,W2,W3,W4),refined(original(F_Persons,F_Safety,F_Buying,F_Maint),id(Z1,Z2,Z3,Z4),counterfactual(Persons,Safety,Buying,Maint),weights(W1,W2,W3,W4),X,cost(O_Persons,O_Safety,O_Buying,O_Maint)).

	ANSWER:	1 (in 3.532 ms)

MODEL:
{  }
BINDINGS: 
F_Persons = 2 
F_Safety = med 
F_Buying = high 
F_Maint = high 
Z1 = 1 
Z2 = 1 
Z3 = 1 
Z4 = 1 
W1 = 1 
W2 = 1 
W3 = 1 
W4 = 1 
Persons = more 
Safety = high 
Buying = vhigh 
Maint = low 
O_Persons = 1 
O_Safety = 1 
O_Buying = 1 
O_Maint = 1


	ANSWER:	2 (in 1.861 ms)

MODEL:
{  }
BINDINGS: 
F_Persons = 2 
F_Safety = med 
F_Buying = high 
F_Maint = high 
Z1 = 1 
Z2 = 1 
Z3 = 1 
Z4 = 1 
W1 = 1 
W2 = 1 
W3 = 1 
W4 = 1 
Persons = more 
Safety = high 
Buying = vhigh 
Maint = med 
O_Persons = 1 
O_Safety = 1 
O_Buying = 1 
O_Maint = 1


	ANSWER:	3 (in 3.296 ms)

MODEL:
{  }
BINDINGS: 
F_Persons = 2 
F_Safety = med 
F_Buying = high 
F_Maint = high 


In [401]:
# We create and save out counterfactual dataframe 
from create_cfs import *
# Counterfactual dataframe anme
cf_df_name = model+'_cars_counterfactuals.csv'

# Create the DataFrame of counterfactuals
instances = create_dataframe_from_textfile(scasp_path+scasp_opt_file)

In [402]:
instances

['F_Persons = 2 \nF_Safety = med \nF_Buying = high \nF_Maint = high \nZ1 = 1 \nZ2 = 1 \nZ3 = 1 \nZ4 = 1 \nW1 = 1 \nW2 = 1 \nW3 = 1 \nW4 = 1 \nPersons = more \nSafety = high \nBuying = vhigh \nMaint = low \nX = X \nO_Persons = 1 \nO_Safety = 1 \nO_Buying = 1 \nO_Maint = 1\n',
 ' \nF_Persons = 2 \nF_Safety = med \nF_Buying = high \nF_Maint = high \nZ1 = 1 \nZ2 = 1 \nZ3 = 1 \nZ4 = 1 \nW1 = 1 \nW2 = 1 \nW3 = 1 \nW4 = 1 \nPersons = more \nSafety = high \nBuying = vhigh \nMaint = med \nX = X \nO_Persons = 1 \nO_Safety = 1 \nO_Buying = 1 \nO_Maint = 1\n',
 ' \nF_Persons = 2 \nF_Safety = med \nF_Buying = high \nF_Maint = high \nZ1 = 1 \nZ2 = 1 \nZ3 = 1 \nZ4 = 1 \nW1 = 1 \nW2 = 1 \nW3 = 1 \nW4 = 1 \nPersons = more \nSafety = high \nBuying = med \nMaint = vhigh \nX = X \nO_Persons = 1 \nO_Safety = 1 \nO_Buying = 1 \nO_Maint = 1\n',
 ' \nF_Persons = 2 \nF_Safety = med \nF_Buying = high \nF_Maint = high \nZ1 = 1 \nZ2 = 1 \nZ3 = 1 \nZ4 = 1 \nW1 = 1 \nW2 = 1 \nW3 = 1 \nW4 = 1 \nPersons = more \nSafe

In [403]:
len(instances)

52

In [404]:
# Identifying the Features of Interest and the Features corresponding to the weight set by the s(CASP) file
# s(CASP) will set the weight of causally altered features to 0 as they have 0 cost
print("We know the feature columns are: ",Feature_Columns)
columns_of_interest = Feature_Columns.copy()
Output_Weight_Columns = []

for i in Feature_Columns:
    Output_Weight_Columns.append('O_'+i)
    columns_of_interest.append('O_'+i)

We know the feature columns are:  ['Persons', 'Safety', 'Buying', 'Maint']


In [405]:
# We add a column CF_id to identify what counterfactual of the s(CASP) file a particular instance in the counterfactual dataframe belongs to
sCASP_df_columns = columns_of_interest.copy()
sCASP_df_columns.append('CF_id')
sCASP_df_columns

['Persons',
 'Safety',
 'Buying',
 'Maint',
 'O_Persons',
 'O_Safety',
 'O_Buying',
 'O_Maint',
 'CF_id']

In [406]:
# We already have the domain information about the desired column
print("Desired Domain info: ",info_dict)

Desired Domain info:  {'Persons': ['more', '4', '2'], 'Safety': ['high', 'med', 'low'], 'Buying': ['vhigh', 'med', 'low', 'high'], 'Maint': ['vhigh', 'low', 'med', 'high']}


In [407]:
# We map the domain information to the respective columns for numeric features
# Setting the domains of the numeric features as s(CASP) returns ranges
candidate_map = {
    #'Duration_months': info_dict['Duration_months']['distinct_values']#,
    #,'Credit_amount': info_dict['Credit_amount']['distinct_values']
}


In [408]:
len(candidate_map)
type(candidate_map)
#candidate_map['Duration_months']
#candidate_map['Credit_amount']

dict

In [409]:
df_org.dtypes

Buying     object
Maint      object
Doors      object
Persons    object
Lugboot    object
Safety     object
Label      object
dtype: object

In [410]:
candidate_map

{}

In [411]:
df_ref.iloc[ref_index]

Buying             high
Maint              high
Doors      morethanfive
Persons               2
Lugboot             big
Safety              med
Label          negative
Name: 2, dtype: object

In [412]:
# We create and save out counterfactual dataframe 
from create_cfs import *

empty_df = pd.DataFrame(columns=sCASP_df_columns)

for i in range(len(instances)):
    parsed_dict = parse_constraints(instances[i])
    print(parsed_dict)
    #print("Hi")
    df_tmp = expand_to_dataframe_k(parsed_dict, columns_of_interest, candidate_map,df_ref.iloc[ref_index], k)
    # Add a column with the same value for all rows
    df_tmp['CF_id'] = i+1
    empty_df = pd.concat([empty_df, df_tmp], ignore_index=True)
    #break

# Remove the last row as there is a ; in cf.txt
empty_df.to_csv('./Counterfactuals/'+cf_df_name)


{'F_Persons': '2', 'F_Safety': 'med', 'F_Buying': 'high', 'F_Maint': 'high', 'Z1': '1', 'Z2': '1', 'Z3': '1', 'Z4': '1', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'Persons': 'more', 'Safety': 'high', 'Buying': 'vhigh', 'Maint': 'low', 'X': 'X', 'O_Persons': '1', 'O_Safety': '1', 'O_Buying': '1', 'O_Maint': '1'}
{'F_Persons': '2', 'F_Safety': 'med', 'F_Buying': 'high', 'F_Maint': 'high', 'Z1': '1', 'Z2': '1', 'Z3': '1', 'Z4': '1', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'Persons': 'more', 'Safety': 'high', 'Buying': 'vhigh', 'Maint': 'med', 'X': 'X', 'O_Persons': '1', 'O_Safety': '1', 'O_Buying': '1', 'O_Maint': '1'}
{'F_Persons': '2', 'F_Safety': 'med', 'F_Buying': 'high', 'F_Maint': 'high', 'Z1': '1', 'Z2': '1', 'Z3': '1', 'Z4': '1', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'Persons': 'more', 'Safety': 'high', 'Buying': 'med', 'Maint': 'vhigh', 'X': 'X', 'O_Persons': '1', 'O_Safety': '1', 'O_Buying': '1', 'O_Maint': '1'}
{'F_Persons': '2', 'F_Safety': 'med', 'F_Buying': 'high

In [413]:
k

20

In [414]:
df_tmp

,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
0,4,med,high,high,1,0,0,0,52


In [415]:
print(parsed_dict)


{'F_Persons': '2', 'F_Safety': 'med', 'F_Buying': 'high', 'F_Maint': 'high', 'Z1': '1', 'Z2': '0', 'Z3': '0', 'Z4': '0', 'W1': '1', 'W2': '1', 'W3': '1', 'W4': '1', 'Persons': '4', 'Safety': 'med', 'Buying': 'high', 'Maint': 'high', 'X': 'X', 'O_Persons': '1', 'O_Safety': '0', 'O_Buying': '0', 'O_Maint': '0'}


# 03

In [416]:
empty_df

,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
0,more,high,vhigh,low,1,1,1,1,1
1,more,high,vhigh,med,1,1,1,1,2
2,more,high,med,vhigh,1,1,1,1,3
3,more,high,med,low,1,1,1,1,4
4,more,high,med,med,1,1,1,1,5
5,more,high,med,high,1,1,1,0,6
6,more,high,low,vhigh,1,1,1,1,7
7,more,high,low,low,1,1,1,1,8
8,more,high,low,med,1,1,1,1,9
9,more,high,low,high,1,1,1,0,10


In [417]:
from zscorescaler import *
from get_dist import *

In [418]:
df_cf = pd.read_csv('./Counterfactuals/'+cf_df_name)
lst_numeric_features = [ 'Duration_months', 'Credit_amount']



k_smallest_cf_df_file_name = model+'_german_'+str(k)+'_smallest_'+norm+'.csv'

df_ref

,Buying,Maint,Doors,Persons,Lugboot,Safety,Label
0,high,high,4,2,med,high,negative
1,med,low,2,4,med,med,positive
2,high,high,morethanfive,2,big,med,negative
3,low,high,morethanfive,4,med,med,positive
4,med,low,4,more,med,low,negative
...,...,...,...,...,...,...,...
341,vhigh,vhigh,morethanfive,more,small,med,negative
342,vhigh,med,4,2,med,med,negative
343,med,low,2,more,small,low,negative
344,vhigh,vhigh,morethanfive,more,small,high,negative


In [419]:
df_ref
df_tmp_ref = df_ref.drop(columns = 'Label')
df_tmp_ref_2 = df_tmp_ref.copy()[Feature_Columns]
df_tmp_ref_2

,Persons,Safety,Buying,Maint
0,2,high,high,high
1,4,med,med,low
2,2,med,high,high
3,4,med,low,high
4,more,low,med,low
...,...,...,...,...
341,more,med,vhigh,vhigh
342,2,med,vhigh,med
343,more,low,med,low
344,more,high,vhigh,vhigh


In [420]:
df_original_sample = df_tmp_ref_2.iloc[[ref_index]]
df_original_sample

,Persons,Safety,Buying,Maint
2,2,med,high,high


In [421]:
# Perform Z-Score Scaling for train set
scaler = ZScoreScaler(features_to_scale=lst_numeric_features)
scaler.fit(df_org)

In [422]:
# Scale the original_sample
df_original_sample_2 = scaler.transform(df_original_sample)
df_original_sample_2

,Persons,Safety,Buying,Maint
2,2,med,high,high


In [423]:
df_cf

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
0,0,more,high,vhigh,low,1,1,1,1,1
1,1,more,high,vhigh,med,1,1,1,1,2
2,2,more,high,med,vhigh,1,1,1,1,3
3,3,more,high,med,low,1,1,1,1,4
4,4,more,high,med,med,1,1,1,1,5
5,5,more,high,med,high,1,1,1,0,6
6,6,more,high,low,vhigh,1,1,1,1,7
7,7,more,high,low,low,1,1,1,1,8
8,8,more,high,low,med,1,1,1,1,9
9,9,more,high,low,high,1,1,1,0,10


In [424]:
# Scale the counterfactuals for distance measure
df_cf_2 = scaler.transform(df_cf)
df_cf_2

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
0,0,more,high,vhigh,low,1,1,1,1,1
1,1,more,high,vhigh,med,1,1,1,1,2
2,2,more,high,med,vhigh,1,1,1,1,3
3,3,more,high,med,low,1,1,1,1,4
4,4,more,high,med,med,1,1,1,1,5
5,5,more,high,med,high,1,1,1,0,6
6,6,more,high,low,vhigh,1,1,1,1,7
7,7,more,high,low,low,1,1,1,1,8
8,8,more,high,low,med,1,1,1,1,9
9,9,more,high,low,high,1,1,1,0,10


In [425]:
# Get the type of each feature and store in a list
feature_types = infer_feature_types(df_original_sample_2)
print(feature_types)


{'Persons': 'categorical', 'Safety': 'categorical', 'Buying': 'categorical', 'Maint': 'categorical'}


In [426]:
# reference_row: original sample that we compare the counterfactuals to for distance
reference_row = df_original_sample_2.iloc[0]
reference_row

Persons       2
Safety      med
Buying     high
Maint      high
Name: 2, dtype: object

In [427]:
df_cf_2.copy()

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
0,0,more,high,vhigh,low,1,1,1,1,1
1,1,more,high,vhigh,med,1,1,1,1,2
2,2,more,high,med,vhigh,1,1,1,1,3
3,3,more,high,med,low,1,1,1,1,4
4,4,more,high,med,med,1,1,1,1,5
5,5,more,high,med,high,1,1,1,0,6
6,6,more,high,low,vhigh,1,1,1,1,7
7,7,more,high,low,low,1,1,1,1,8
8,8,more,high,low,med,1,1,1,1,9
9,9,more,high,low,high,1,1,1,0,10


In [428]:
Feature_Columns

['Persons', 'Safety', 'Buying', 'Maint']

In [429]:
df_cf_2

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
0,0,more,high,vhigh,low,1,1,1,1,1
1,1,more,high,vhigh,med,1,1,1,1,2
2,2,more,high,med,vhigh,1,1,1,1,3
3,3,more,high,med,low,1,1,1,1,4
4,4,more,high,med,med,1,1,1,1,5
5,5,more,high,med,high,1,1,1,0,6
6,6,more,high,low,vhigh,1,1,1,1,7
7,7,more,high,low,low,1,1,1,1,8
8,8,more,high,low,med,1,1,1,1,9
9,9,more,high,low,high,1,1,1,0,10


In [430]:
# Get a df of disances of each cf compared to the reference row, i.e., the original sample (scaled)
#df_cf_distances = add_distance_columns1(Feature_Columns, df_cf_2.copy(), reference_row, feature_types)
df_cf_distances = add_distance_columns_vectorized(Feature_Columns, df_cf_2.copy(), reference_row, feature_types)
df_cf_distances

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id,L0,L1,L2
0,0,more,high,vhigh,low,1,1,1,1,1,4,4,2.000000
1,1,more,high,vhigh,med,1,1,1,1,2,4,4,2.000000
2,2,more,high,med,vhigh,1,1,1,1,3,4,4,2.000000
3,3,more,high,med,low,1,1,1,1,4,4,4,2.000000
4,4,more,high,med,med,1,1,1,1,5,4,4,2.000000
5,5,more,high,med,high,1,1,1,0,6,3,3,1.732051
6,6,more,high,low,vhigh,1,1,1,1,7,4,4,2.000000
7,7,more,high,low,low,1,1,1,1,8,4,4,2.000000
8,8,more,high,low,med,1,1,1,1,9,4,4,2.000000
9,9,more,high,low,high,1,1,1,0,10,3,3,1.732051


In [431]:
k

20

In [432]:

k_smallest_cf_df_file_name = 'vector_'+model+'_german_'+str(k)+'_smallest_'+norm+'.csv'


In [433]:
# Select the k nearest counterfactuals as per the norm
df_k_smallest,  k_smallest_indices = get_k_smallest_points(df_cf_distances, distance_col=norm, k=k)
print(df_k_smallest)


    Unnamed: 0 Persons Safety Buying  Maint  O_Persons  O_Safety  O_Buying  \
25          25    more    med   high   high          1         0         0   
51          51       4    med   high   high          1         0         0   
49          49       4    med   high    low          1         0         0   
48          48       4    med    low   high          1         0         1   
18          18    more    med    med   high          1         0         1   
38          38       4   high   high   high          1         1         0   
23          23    more    med   high    low          1         0         0   
24          24    more    med   high    med          1         0         0   
50          50       4    med   high    med          1         0         0   
44          44       4    med    med   high          1         0         1   
22          22    more    med    low   high          1         0         1   
12          12    more   high   high   high          1         1

In [434]:
df_fact.iloc[ref_index]

Persons       2
Safety      med
Buying     high
Maint      high
Name: 2, dtype: object

In [435]:
df_k_smallest

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id,L0,L1,L2
25,25,more,med,high,high,1,0,0,0,26,1,1,1.000000
51,51,4,med,high,high,1,0,0,0,52,1,1,1.000000
49,49,4,med,high,low,1,0,0,1,50,2,2,1.414214
48,48,4,med,low,high,1,0,1,0,49,2,2,1.414214
18,18,more,med,med,high,1,0,1,0,19,2,2,1.414214
38,38,4,high,high,high,1,1,0,0,39,2,2,1.414214
23,23,more,med,high,low,1,0,0,1,24,2,2,1.414214
24,24,more,med,high,med,1,0,0,1,25,2,2,1.414214
50,50,4,med,high,med,1,0,0,1,51,2,2,1.414214
44,44,4,med,med,high,1,0,1,0,45,2,2,1.414214


In [436]:
df_k_smallest
k_smallest_indices
df_cf.iloc[k_smallest_indices]

,Unnamed: 0,Persons,Safety,Buying,Maint,O_Persons,O_Safety,O_Buying,O_Maint,CF_id
25,25,more,med,high,high,1,0,0,0,26
51,51,4,med,high,high,1,0,0,0,52
49,49,4,med,high,low,1,0,0,1,50
48,48,4,med,low,high,1,0,1,0,49
18,18,more,med,med,high,1,0,1,0,19
38,38,4,high,high,high,1,1,0,0,39
23,23,more,med,high,low,1,0,0,1,24
24,24,more,med,high,med,1,0,0,1,25
50,50,4,med,high,med,1,0,0,1,51
44,44,4,med,med,high,1,0,1,0,45


In [437]:
# Obtain the complete list of k smallest counterfactuals with all the columns
df_cf_final = df_cf.iloc[k_smallest_indices][Feature_Columns].copy()
df_cf_final

,Persons,Safety,Buying,Maint
25,more,med,high,high
51,4,med,high,high
49,4,med,high,low
48,4,med,low,high
18,more,med,med,high
38,4,high,high,high
23,more,med,high,low
24,more,med,high,med
50,4,med,high,med
44,4,med,med,high


In [438]:
# 1) Identify extra columns:
extra_cols = set(df_tmp_ref.columns) - set(df_cf_final.columns)
# e.g. {'Extra1', 'Extra2'}

# 2) Extract the SINGLE instance from df_B
#    (assuming df_B really has only one row, or you specifically want the 0th row)
row_b = df_tmp_ref.iloc[0]  # This is a pandas Series representing that row.

# 3) For each extra column, set all rows of df_A to the single value from df_B
for col in extra_cols:
    df_cf_final[col] = row_b[col]

# Now df_A has the extra columns, each row filled with the single-instance value from df_B
print(df_cf_final)

   Persons Safety Buying  Maint Lugboot Doors
25    more    med   high   high     med     4
51       4    med   high   high     med     4
49       4    med   high    low     med     4
48       4    med    low   high     med     4
18    more    med    med   high     med     4
38       4   high   high   high     med     4
23    more    med   high    low     med     4
24    more    med   high    med     med     4
50       4    med   high    med     med     4
44       4    med    med   high     med     4
22    more    med    low   high     med     4
12    more   high   high   high     med     4
19    more    med    low  vhigh     med     4
42       4    med    med    low     med     4
41       4    med    med  vhigh     med     4
40       4    med  vhigh    med     med     4
31       4   high    med   high     med     4
39       4    med  vhigh    low     med     4
37       4   high   high    med     med     4
43       4    med    med    med     med     4


In [439]:
df_cf_final

,Persons,Safety,Buying,Maint,Lugboot,Doors
25,more,med,high,high,med,4
51,4,med,high,high,med,4
49,4,med,high,low,med,4
48,4,med,low,high,med,4
18,more,med,med,high,med,4
38,4,high,high,high,med,4
23,more,med,high,low,med,4
24,more,med,high,med,med,4
50,4,med,high,med,med,4
44,4,med,med,high,med,4


In [440]:
# Add distances to the counterfactual dataframe
# Specify the columns to add from df_B
columns_to_add = ['L0', 'L1', 'L2']  # Replace with actual column names

# Add them to df_A
df_cf_final[columns_to_add] = df_k_smallest[columns_to_add]

In [441]:
df_cf_final
df_cf_final.head(k)

,Persons,Safety,Buying,Maint,Lugboot,Doors,L0,L1,L2
25,more,med,high,high,med,4,1,1,1.000000
51,4,med,high,high,med,4,1,1,1.000000
49,4,med,high,low,med,4,2,2,1.414214
48,4,med,low,high,med,4,2,2,1.414214
18,more,med,med,high,med,4,2,2,1.414214
38,4,high,high,high,med,4,2,2,1.414214
23,more,med,high,low,med,4,2,2,1.414214
24,more,med,high,med,med,4,2,2,1.414214
50,4,med,high,med,med,4,2,2,1.414214
44,4,med,med,high,med,4,2,2,1.414214


In [442]:
df_cf_final.to_csv('./Counterfactuals/'+k_smallest_cf_df_file_name)